In [206]:
import pandas as pd
import networkx as nx
from tqdm import tqdm
import pickle
import itertools
from itertools import permutations
import folium
import numpy as np
import decimal
from  more_itertools import unique_everseen


from collections import defaultdict
from tqdm import tqdm_notebook

## Organizing pandas

### 1. Coordinates (Node information file)

In [93]:
coordinates = pd.read_csv(r"C:\Users\Luca\Desktop\WIP\ADM HW 5\data\USA-road-d.CAL.co", sep=' ', delimiter=None,
                      index_col=None, usecols=None, encoding='ISO-8859-1')

C:\Users\Luca\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (1,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [94]:
# combining two columns pandas
coordinates['combined']=coordinates['c'].astype(str)+''+coordinates['9th'].astype(str)

In [95]:
coordinates = coordinates.iloc[6:] # deleting the first 5 rows

In [96]:
# organizing pandas DF
coordinates.drop(['c', '9th', 'Challenge:', 'Shortest', 'Paths'], axis=1, inplace=True)

In [97]:
coordinates.columns = ['Latitude', 'Longitude', 'Id_Node']

In [98]:
coordinates = coordinates.reset_index(drop=True)

In [99]:
# Transform all str into int
coordinates[['Latitude', 'Longitude']] = coordinates[['Latitude', 'Longitude']].astype(int)

In [100]:
coordinates.head(20)

,Latitude,Longitude,Id_Node
0,-114315309,34133550,v1
1,-114223946,34176221,v2
2,-114307299,34148791,v3
3,-114318765,34138889,v4
4,-114347300,34042614,v5
5,-114347162,34058931,v6
6,-114126950,33988170,v7
7,-114347268,34028261,v8
8,-114340613,34042519,v9
9,-114324855,34042664,v10


### 2. Distance graph

In [198]:
distance = pd.read_csv(r"data\USA-road-d.CAL.gr", sep=' ', delimiter=None,
                      index_col=None, usecols=None, encoding='ISO-8859-1', error_bad_lines=False)

b'Skipping line 6: expected 7 fields, saw 8\n'
C:\Users\Luca\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [199]:
distance = distance.iloc[6:] # deleting the first 5 rows

In [200]:
# organizing pandas DF
distance.drop(['c','Challenge:', 'Shortest', 'Paths'], axis=1, inplace=True)

(Id_Node1, Id_Node2, d(Id_Node1,Id_Node2)), where d(x,y) is the physical distance between x and y.

In [201]:
distance.columns = ['Id_Node1', 'Id_Node2', 'Distance']

In [202]:
distance = distance.reset_index(drop=True)

In [203]:
# Transform all str into int
distance = distance.astype(int)

In [204]:
distance.head(20)

,Id_Node1,Id_Node2,Distance
0,1048577,1,456
1,2,1048578,2389
2,1048578,2,2389
3,3,1048579,358
4,1048579,3,358
5,4,1048579,14901
6,1048579,4,14901
7,5,6,18133
8,6,5,18133
9,7,1048580,2463


### 3. Time travel graph

In [8]:
travel = pd.read_csv(r"data\USA-road-t.CAL.gr", sep=' ', delimiter=None,
                      index_col=None, usecols=None, encoding='ISO-8859-1', error_bad_lines = False)

b'Skipping line 6: expected 7 fields, saw 8\n'
C:\Users\Luca\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
travel = travel.iloc[6:] # deleting the first 5 rows

In [10]:
# organizing pandas DF
travel.drop(['c','Challenge:', 'Shortest', 'Paths'], axis=1, inplace=True)

(Node1, Node2, t(Id_Node1, Id_Node2))

In [11]:
travel.columns = ['Id_Node1', 'Id_Node2', 'Time distance']

In [12]:
travel = travel.reset_index(drop=True)

In [13]:
travel.head(20)

,Id_Node1,Id_Node2,Time distance
0,1048577,1,1139
1,2,1048578,5972
2,1048578,2,5972
3,3,1048579,895
4,1048579,3,895
5,4,1048579,37253
6,1048579,4,37253
7,5,6,45332
8,6,5,45332
9,7,1048580,6158


In [14]:
# Transform all str into int
travel = travel.astype(int)

## Part 1. Create data types for nodes and edges

In [ ]:
# Create a list of all nodes
nodes =  []
nodes.append([i for i in range (1,1890816)])

In [ ]:
# Create a list of set with (Node_1, Node_2)
edges = []

In [ ]:
temp_travel = travel.drop(['Time distance'], axis=1)
for index, element in tqdm(temp_travel.iterrows()):
    edge = [] # couples of edges
    edge.extend([element[0], element[1]])
    edges.append(edge)

In [ ]:
edges

We have to find out if all verteces are bidirectional, to state that it's a bidirectional or directional graph. <br/>
Since the data downloaded talkes about Arcs instead of Edges we can assume that it's a directed graph

In [ ]:
with open('outfile', 'wb') as fp:
    pickle.dump(edges, fp)

In [62]:
with open ('outfile', 'rb') as fp:
    edges = pickle.load(fp)

### Create the adjacency dictionary

In [208]:
adj_dic = {}

In [ ]:
for i in tqdm(edges):
    if i[0] not in adj_dic: # check if first node is a key already 
        
        adj_dic[i[0]] = []
        adj_dic[i[0]].append(i[1])
    
    else:
        
        if i[1] in adj_dic[i[0]]:
            continue
        else:
            adj_dic[i[0]].append(i[1])  # if True check if second node is in dictionary, if not append
    
    if i[1] not in adj_dic: # check if first node is a key already
        
        adj_dic[i[1]] = []
        adj_dic[i[1]].append(i[0])
    
    else:
        
        if i[0] in adj_dic[i[1]]:
            continue
        else:
            adj_dic[i[1]].append(i[0])  # if True check if second node is in dictionary, if not append

In [ ]:
# we saved this dictionary to a picke file so that we don't have to run the code each time
with open('adj_dic', 'wb') as fp:
    pickle.dump(adj_dic, fp)

In [210]:
with open('adj_dic', 'rb') as fp:
    adj_dic = pickle.load(fp)

### Weighted adjacency matrix

#### 1. Time 

In [ ]:
graph_trav_dist = defaultdict(list)
for row in tqdm_notebook(travel.iterrows()):
    if int(row[1]["Id_Node2"]) in adj_dic[int(row[1]["Id_Node1"])]:
        graph_trav_dist[int(row[1]["Id_Node1"])].append((int(row[1]["Id_Node2"]),int(row[1]["Time distance"])))
        

In [ ]:
# we saved this dictionary to a picke file so that we don't have to run the code each time
with open('adj_dic_travel', 'wb') as fp:
    pickle.dump(graph_trav_dist,fp)

#### 2. Distance

In [ ]:
graph_ph_dist = defaultdict(list)
for row in tqdm_notebook(distance.iterrows()):
    if int(row[1]["Id_Node2"]) in adj_dic[int(row[1]["Id_Node1"])]:
        graph_ph_dist[int(row[1]["Id_Node1"])].append((int(row[1]["Id_Node2"]),int(row[1]["Distance"])))
        

In [ ]:
# we saved this dictionary to a picke file so that we don't have to run the code each time
with open('adj_dic_ph', 'wb') as fp:
    pickle.dump(graph_ph_dist,fp)

#### 3. Network

In [ ]:
graph_net_dist = defaultdict(list)
for row in tqdm_notebook(distance.iterrows()):
    if int(row[1]["Id_Node2"]) in adj_dic[int(row[1]["Id_Node1"])]:
        graph_ph_dist[int(row[1]["Id_Node1"])].append((int(row[1]["Id_Node2"]), 1)
        

In [ ]:
# we saved this dictionary to a picke file so that we don't have to run the code each time
with open('adj_dic_net', 'wb') as fp:
    pickle.dump(graph_ph_dist,fp)

## 2.Time travel

In [30]:
with open('adj_dic_travel', 'rb') as fp:
    graph_trav_dist = pickle.load(fp)

In [103]:
set_of_nodes = input()

1 2 3


In [102]:
set_of_nodes = set_of_nodes.split(" ")
perm = permutations(set_of_nodes, 2)

In [34]:
nodes = list(graph_trav_dist.keys())
distances = graph_trav_dist

all_distances = {}

for p in perm:
    print(p)
    unvisited = {node: None for node in nodes} #using None as +inf
    visited = {}
    current = int(p[0])
    end = int(p[1])
    currentDistance = 0
    unvisited[current] = currentDistance

    while True:
        for neighbour, distance in distances[current]:
            if neighbour not in unvisited: continue
            newDistance = currentDistance + distance
            if unvisited[neighbour] is None or unvisited[neighbour] > newDistance:
                unvisited[neighbour] = newDistance
        visited[current] = currentDistance
        del unvisited[current]
        if not unvisited or current == end: 
            all_distances[p] = currentDistance
            break
        candidates = [node for node in unvisited.items() if node[1]]
        current, currentDistance = sorted(candidates, key = lambda x: x[1])[0]
        

('1', '3')
('2', '1')
('2', '3')
('3', '1')
('3', '2')


In [35]:
candidates = []
for pair1 in all_distances:
    for pair2 in all_distances:
        if pair1[1] == pair2[0] and pair1[0] != pair2[1]:
            candidates.append((pair1, pair2))

In [45]:
p = permutations(candidates, len(candidates)-1)

In [46]:
permu = [i for i in p]

In [ ]:
res = []
for i in permu:
    
    for z in i:
        trues = []
        print(z)
        for j in range(len(candidates)-3):
            print(z[j])
            trues.append(z[j][1]==z[j+1][0])
        if all([l == True for l in trues] ):
            res.append(z)
            

In [ ]:
distances = []
for tuple_ in res[:-1]:
    try:
        distances.append((all_distances[tuple_[0]] + all_distances[tuple_[1]]))
    except:
        pass

In [185]:
# the best result!
best = candidates[distances.index(min(distances))]

In [186]:
b = [i for sublist in best for i in sublist]

In [187]:
b = list(unique_everseen(b))

In [188]:
b

['2', '3', '1']

## Plot 

In [189]:
map = folium.Map(location = [34.133550,-114.315309], zoom_start = 6)

In [193]:
coor = []
for node in b:
    
    longitude = str(coordinates.iloc[int(node)-1]['Longitude'])
    longitude = decimal.Decimal(longitude[:2] + "." + longitude[2:])
    
    latitude = str(coordinates.iloc[int(node)-1]['Latitude'])
    latitude = decimal.Decimal(latitude[:4] + "." + latitude[4:])
    
    folium.Marker([longitude, latitude]).add_to(map)
    coor.append((longitude, latitude))


In [194]:
folium.PolyLine(coor).add_to(map)

In [195]:
map

## 3. Shortest Ordered Route

We'll use Dijkstra algorithm to find the shortest paths

### Best Idea

In [ ]:
set_of_nodes = list(map(int, input().split(' ')))

In [ ]:
def walk(graph, set_of_nodes, all_distances = {}):
    
    print(set_of_nodes)
    nodes = list(graph.keys())
    distances = graph
    
    if len(set_of_nodes) == 1:
        return all_distances
    
    for i in range(len(set_of_nodes)-1):
        unvisited = {node: None for node in nodes} #using None as +inf
        visited = {}
        current = set_of_nodes[0]
        end = set_of_nodes[1]
        currentDistance = 0
        unvisited[current] = currentDistance
        
        while True:
            for neighbour, distance in distances[current]:
                if neighbour not in unvisited: continue
                newDistance = currentDistance + distance
                if unvisited[neighbour] is None or unvisited[neighbour] > newDistance:
                    unvisited[neighbour] = newDistance
            visited[current] = currentDistance
            del unvisited[current]
            
            if current == end: 
                print('Successful')
                all_distances["{} -> {}".format(set_of_nodes[i], end)] = visited
                set_of_nodes.pop(0)
                walk(graph, set_of_nodes, all_distances)
                break
            
            if not unvisited:
                print('Not possible')
                break
            
            if current in set_of_nodes[2:]:
                print('Not ordered')
                del graph[current]
                for v in graph.values():
                    for i in v:
                        if i[0] == current:
                            v.remove(i)               
                walk(graph, set_of_nodes, all_distances)
            
            candidates = [node for node in unvisited.items() if node[1]]
            current, currentDistance = sorted(candidates, key = lambda x: x[1])[0]


In [ ]:
w = walk(graph_trav_dist, set_of_nodes, {})

## Bad Ideas

### Create Dijkstra

In [18]:
import itertools

In [2]:
graph = {1:{2:10, 3:3},2 :{3:1, 4:2},3:{2:4, 4:8, 5:2},4:{5:7},5:{4:9}}

In [3]:
shortest_distance = {}
predecessor = {}
unvisited = graph.copy()
path = []
for node in unvisited:
    shortest_distance[node] = float('inf')
shortest_distance[1] = 0

In [6]:
while unvisited:
    
    minNode = None
    for node in unvisited:
        if minNode is None: # for the first case, when it's empty
            minNode = node
        elif shortest_distance[node] < shortest_distance[minNode]:
            minNode = node
            
    # main part of the algo
    for childNode, weight in graph[minNode].items():
        if weight + shortest_distance[minNode] < shortest_distance[childNode]:
            shortest_distance[childNode] = weight + shortest_distance[minNode]
            predecessor[childNode] = minNode

    unvisited.pop(minNode)

In [7]:
shortest_distance

{1: 0, 2: 7, 3: 3, 4: 9, 5: 5}

In [8]:
predecessor

{2: 3, 3: 1, 4: 2, 5: 3}

### Create path

In [19]:
def find_path(graph, start, end, path=[]):
        path = path + [start]
        print("st:%s, end:%s, path:%s" % (start,end,path))
        if (start == end) or (end in graph[start]) :
            return path
        if start not in graph:
            return None
        print("available routes: %s" % graph[start])
        for node in graph[start]:
            if node not in path:
                print("next node: %s" % node)
                newpath = find_path(graph, node, end, path)
                if newpath: return newpath
        return None

In [20]:
visit = [2, 5, 4]
visit = visit[::-1] # we reverse the list of nodes we want to visit

In [21]:
megapath = [] # entire path that we want

In [22]:
dis = 0 

In [23]:
while len(visit) != 1:
    path = [] # for each iteration
    start = visit[1] # second element of the list
    goal = visit[0] # first element of the lsit
    currentNode = goal # we go backwards, we want to find the predecessor
    while currentNode != start:
        
        try: # if we can find the predecessor this is what we do
            path.insert(0, currentNode)
            currentNode = predecessor[currentNode]
            print('first')
        
        
        except KeyError: # if there is not any predecessor
            
            if goal in graph[start]: # if goal is a neighbour of start 
                path = [] # path is empty
                path.insert(0, goal) # we use this as path
                dis += graph[start][goal] # and update distance
                print('here')
            
            else : 
                path = []
                path = find_path(graph, start, goal, path)
                path.insert(len(path),goal) # add the end to the list
                print('Path not reachable')
            break
   
    megapath.insert(0, path)
    visit.pop(0)

first
first
first
here
first
first
st:2, end:5, path:[2]
available routes: {3: 1, 4: 2}
next node: 3
st:3, end:5, path:[2, 3]
Path not reachable


In [24]:
merged = list(itertools.chain.from_iterable(megapath))

In [25]:
if merged[0] != start:
    merged.insert(0,start)   

In [26]:
merged

[2, 3, 5, 4]

### Test with 2 nodes

In [470]:
total_dis = 0

In [18]:
path = []
dis = 0
start = 2
goal = 5

In [20]:
currentNode = goal
while currentNode != start:
    try:
        path.insert(0, currentNode)
        currentNode = predecessor[currentNode]
        dis = shortest_distance[goal] 
        print('first')
    except KeyError:
        
       
        if goal in graph[start]:
            path = []
            dis = 0
            path.insert(0, goal)
            dis = graph[start][goal]
            print('here')
        else : 
            path = []
            path = find_path(graph, start, goal, path)
            path.insert(len(path),goal) # add the end to the list
            print('Path not reachable')
        break
path.insert(0,start)   

first
first
st:2, end:5, path:[2]
available routes: {3: 1, 4: 2}
next node: 3
st:3, end:5, path:[2, 3]
Path not reachable


In [520]:
predecessor

{2: 3, 3: 1, 4: 2, 5: 3}

In [517]:
shortest_distance

{1: 0, 2: 7, 3: 3, 4: 9, 5: 5}

In [21]:
path

[2, 2, 3, 5]

In [471]:
total_dis += dis

In [472]:
total_dis

1

In [484]:
if shortest_distance[goal] != float('inf'):
    print('shortest distance is', str(shortest_distance[goal]))
    print('and the path is'+ str(path))

shortest distance is 5
and the path is[1, 3, 5]


### Step-by Step

In [461]:
path  = []

In [462]:
visit

[5, 3, 2]

In [463]:
start = visit[1]
goal = visit[0]

In [464]:
start, goal

(3, 5)

In [446]:
currentNode = goal
while currentNode != start:
    try:
        path.insert(0, currentNode)
        currentNode = predecessor[currentNode]
        print('first')
    except KeyError:
        
        path = []
        if goal in graph[start]:
            path.insert(0, goal)
            dis += graph[start][goal]
            print('here')
        else : 
            path = []
            print('Path not reachable')
        break
megapath.insert(0, path)
visit.pop(0)

first
here


In [ ]:
if shortest_distance[goal] != float('inf'):
    print('shortest distance is', shortest_distance[goal]+dis)
    print('and the path is'+ str(path))

In [448]:
# when iteration is finished
megapath.insert(0,start)   

In [449]:
path 

[3]

In [450]:
megapath

[2, [3], [5]]

In [577]:
def find_path(graph, start, end, path=[]):
        path = path + [start]
        print("st:%s, end:%s, path:%s" % (start,end,path))
        if (start == end) or (end in graph[start]) :
            return path
        if start not in graph:
            return None
        print("available routes: %s" % graph[start])
        for node in graph[start]:
            if node not in path:
                print("next node: %s" % node)
                newpath = find_path(graph, node, end, path)
                if newpath: return newpath
        return None

In [578]:
p = []
test = find_path(graph, 1, 4, p)
test.insert(len(test),4) # add the end to the list

st:1, end:4, path:[1]
available routes: {2: 10, 3: 3}
next node: 2
st:2, end:4, path:[1, 2]


In [579]:
test

[1, 2, 4]